### 由于Apriori关联规则算法无法处理连续型数值变量，所以使用聚类算法对各个证型系数进行离散化处理
> p_188

In [1]:
import pandas as pd
# 导入KMeans聚类
from sklearn.cluster import KMeans

In [27]:
# 带聚类文件
datafile = 'E:\\NLP\\Python_Data_Analysis_And_Mining_Actual_Combat\\chapter8\\demo\\data\\data.xls'
# 离散化处理之后的保存文件
processedfile = 'E:\\NLP\\Python_Data_Analysis_And_Mining_Actual_Combat\\chapter8\\demo\\tmp\\data_processed_test.xls'

In [3]:
# 各证型系数标签
typelabel = {
    '肝气郁结证型系数': 'A',
    '热毒蕴结证型系数': 'B',
    '冲任失调证型系数': 'C',
    '气血两虚证型系数': 'D',
    '脾胃虚弱证型系数': 'E',
    '肝肾阴虚证型系数': 'F'
}
# 需要聚类的类别
k = 4

In [6]:
# 读取数据
data = pd.read_excel(datafile)
# 各证型系数
keys = list(typelabel.keys())
print(keys)
# 结果储存容器
result = pd.DataFrame()
print(result)
test = pd.Series()
print(test)

['肝气郁结证型系数', '热毒蕴结证型系数', '冲任失调证型系数', '气血两虚证型系数', '脾胃虚弱证型系数', '肝肾阴虚证型系数']
Empty DataFrame
Columns: []
Index: []
Series([], dtype: float64)


In [32]:
# 依次对每个证型系数进行聚类
for i in range(len(keys)):
    # 调用K-Means算法进行聚类离散化
    print('当前进行 %s 的聚类' %(keys[i]))
    # n_jobs = 4 并行数，一般等于cpu数就好
    kmodel = KMeans(n_clusters = k, n_jobs = 4)
    # 训练 (取对应证型系数的那一列并转为矩阵形式)
    kmodel.fit(data[[keys[i]]].as_matrix())
    
    # 聚类中心
    r1 = pd.DataFrame(kmodel.cluster_centers_, columns = [typelabel[keys[i]]])
    # 分类统计 看上面的 test 和 result 打印的结果就明白了
    r2 = pd.Series(kmodel.labels_).value_counts()
    # 再转为DataFrame形式, 记录各个类别的数目
    r2 = pd.DataFrame(r2, columns = [typelabel[keys[i]] + 'n'])
    # 匹配聚类中心和类别数目
    # 以值排序
#     r = pd.concat([r1, r2], axis = 1).sort_values(typelabel[keys[i]])
    r = pd.concat([r1, r2], axis = 1).sort_values(typelabel[keys[i]])
    print(r)
    r.index = [1, 2, 3, 4]
    # rolling_mean() 用来计算相邻 2列 的均值，以此作为边界点
    r[typelabel[keys[i]]] = pd.Series.rolling(r[typelabel[keys[i]]], 2).mean()
    # 将原来的聚类中心改为边界点
    r_copy = r[typelabel[keys[i]]].copy()
    r_copy[1] = 0.0
    result = result.append(r_copy.T)

当前进行 肝气郁结证型系数 的聚类
          A   An
0  0.136954  240
2  0.220441  356
1  0.295007  281
3  0.408679   53
当前进行 热毒蕴结证型系数 的聚类
          B   Bn
0  0.097456  342
2  0.209629  380
1  0.386804  179
3  0.593103   29
当前进行 冲任失调证型系数 的聚类
          C   Cn
0  0.158588  296
3  0.245232  393
1  0.332136  206
2  0.514514   35
当前进行 气血两虚证型系数 的聚类
          D   Dn
0  0.138201  309
2  0.214695  370
1  0.298915  211
3  0.431275   40
当前进行 脾胃虚弱证型系数 的聚类
          E   En
3  0.111396  273
0  0.194000  319
1  0.321747  245
2  0.430376   93
当前进行 肝肾阴虚证型系数 的聚类
          F   Fn
3  0.142080  200
2  0.216207  237
0  0.306566  265
1  0.402719  228


In [28]:
# 以Index排序(A, B, C, D, E, F)
result = result.sort_index()
result.to_excel(processedfile)

<class 'pandas.core.frame.DataFrame'>
